In [3]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
import warnings
from sklearn.exceptions import ConvergenceWarning

# Omitir warnings caso a rede demore a convergir
warnings.filterwarnings("ignore", category=ConvergenceWarning)

def treinar_pmc_customizado(X_train, y_train, seed_aleatoria):
    """
    Treina um Perceptron de Múltiplas Camadas (PMC) inicializando
    todos os pesos propositalmente entre 0 e 1, conforme pedido.
    """
    np.random.seed(seed_aleatoria)

    # Criei o modelo PMC (1 camada oculta com 5 neurônios, função de ativação tangente hiperbólica)
    # A tangente hiperbólica (tanh) é ótima aqui pois mapeia valores para [-1, 1] (suas classes C1 e C2)
    clf = MLPClassifier(hidden_layer_sizes=(5,), activation='tanh', solver='adam',
                        max_iter=1, warm_start=True, random_state=seed_aleatoria)

    # Executei 1 época apenas para o sklearn criar as matrizes de pesos (coefs_ e intercepts_)
    clf.fit(X_train, y_train)

    # SOBRESCREVENDO OS PESOS (Requisito: inicializar entre 0 e 1)
    clf.coefs_ = [np.random.uniform(0, 1, size=w.shape) for w in clf.coefs_]
    clf.intercepts_ = [np.random.uniform(0, 1, size=b.shape) for b in clf.intercepts_]

    # Retomei o treinamento com até 2000 épocas
    clf.set_params(max_iter=2000)
    clf.fit(X_train, y_train) # O fit continuará a partir dos novos pesos (warm_start=True)

    return clf

# 1. CARREGAMENTO DOS DADOS DE TREINAMENTO
try:
    # Tenta ler o arquivo Excel original da aba de treinamento
    df_train = pd.read_excel('dados Trafos1.xlsx', sheet_name='tab_treinamento')
    X_train = df_train[['X1', 'X2', 'X3']].values
    y_train = df_train['d'].values
    print("Sucesso: Arquivo Excel de treinamento carregado!")
except FileNotFoundError:
    print("AVISO: 'dados Trafos1.xlsx' não encontrado.")
    print("Utilizando as amostras do ANEXO (PDF) para fins de demonstração...\n")
    # Amostras do anexo no fim da página 2
    X_train = np.array([
        [-0.6508, 0.1097, 4.0009],
        [-1.4492, 0.8896, 4.4005],
        [2.0149, 0.6192, 10.9263],
        [0.2012, 0.2611, 5.4631]
    ])
    y_train = np.array([-1, -1, -1, 1])

# 2. ITEM A: EXECUTAR 2 TREINAMENTOS
# Seeds diferentes garantem inicializações aleatórias (entre 0 e 1) distintas
pmc_T1 = treinar_pmc_customizado(X_train, y_train, seed_aleatoria=42)
pmc_T2 = treinar_pmc_customizado(X_train, y_train, seed_aleatoria=99)

# 3. ITEM B: REGISTRAR PESOS, BIAS E ÉPOCAS
print("-" * 50)
print("TREINAMENTO 1 (T1):")
print(f"Número de Épocas iteradas: {pmc_T1.n_iter_}")
print("Pesos (Camada de Entrada -> Oculta):\n", pmc_T1.coefs_[0])
print("Bias (Camada Oculta):\n", pmc_T1.intercepts_[0])
print("Pesos (Camada Oculta -> Saída):\n", pmc_T1.coefs_[1])
print("Bias (Saída):\n", pmc_T1.intercepts_[1])

print("-" * 50)
print("TREINAMENTO 2 (T2):")
print(f"Número de Épocas iteradas: {pmc_T2.n_iter_}")
print("Pesos (Camada de Entrada -> Oculta):\n", pmc_T2.coefs_[0])
print("Bias (Camada Oculta):\n", pmc_T2.intercepts_[0])
print("Pesos (Camada Oculta -> Saída):\n", pmc_T2.coefs_[1])
print("Bias (Saída):\n", pmc_T2.intercepts_[1])
print("-" * 50)

# 4. ITEM C: PREVISÃO NO QUADRO DE TESTE
# Dados da tabela presente no enunciado da Questão 5.c
dados_teste = {
    'Amostra': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'X1': [-0.3565, -0.7842, 0.3012, 0.7757, 0.1570, -0.7014, 0.3748, -0.6920, -1.3970, -1.8842],
    'X2': [0.0620, 1.1267, 0.5611, 1.0648, 0.8028, 1.0316, 0.1536, 0.9404, 0.7141, -0.2805],
    'X3': [5.9891, 5.5912, 5.8234, 8.0677, 6.3040, 3.6005, 6.1537, 4.4058, 4.9263, 1.2548]
}

df_teste = pd.DataFrame(dados_teste)
X_teste = df_teste[['X1', 'X2', 'X3']].values

# Realizando as classificações usando os modelos T1 e T2
df_teste['y (T1)'] = pmc_T1.predict(X_teste)
df_teste['y (T2)'] = pmc_T2.predict(X_teste)

print("\nRESULTADOS DO ITEM C - Classificação Automática:")
print(df_teste.to_string(index=False))

AVISO: 'dados Trafos1.xlsx' não encontrado.
Utilizando as amostras do ANEXO (PDF) para fins de demonstração...

--------------------------------------------------
TREINAMENTO 1 (T1):
Número de Épocas iteradas: 11
Pesos (Camada de Entrada -> Oculta):
 [[0.38551946 0.96175615 0.74303618 0.60968786 0.16704824]
 [0.14499879 0.04704235 0.8551356  0.5900831  0.69703599]
 [0.00958261 0.95886985 0.82140245 0.20131231 0.17079689]]
Bias (Camada Oculta):
 [0.60085742 0.12845535 0.28110577 0.35533532 0.44504311]
Pesos (Camada Oculta -> Saída):
 [[0.17244544]
 [0.29324617]
 [0.51376031]
 [0.42096066]
 [0.28023782]]
Bias (Saída):
 [0.77417975]
--------------------------------------------------
TREINAMENTO 2 (T2):
Número de Épocas iteradas: 11
Pesos (Camada de Entrada -> Oculta):
 [[ 0.68332166  0.49912797  0.83652626  0.04244295  0.81910064]
 [ 0.55457314  0.28657108  0.03566463  0.97964041 -0.0042237 ]
 [ 0.75875223  0.73571982  0.36640798  0.483147    0.9178996 ]]
Bias (Camada Oculta):
 [0.2006469

In [ ]:
from google.colab import drive
drive.mount('/content/drive')